In [4]:
import os
import requests
import numpy as np
import pandas as pd

from xgboost import XGBClassifier

from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)


from yellowbrick.features import ParallelCoordinates, Rank2D, JointPlotVisualizer, RadViz, Rank1D
from yellowbrick.model_selection import FeatureImportances
from yellowbrick.classifier import ClassificationReport, ROCAUC, ClassPredictionError
from yellowbrick.cluster import SilhouetteVisualizer

In [5]:
df = pd.read_csv('../titanic_data/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import roc_curve
from sklearn.decomposition import PCA

In [7]:
df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)

In [8]:
df['Cabin_Class'] = df['Cabin'].str[0]
df['Cabin_Class'] = df['Cabin_Class'].fillna('na')

df['Embarked'] = df['Embarked'].astype(str)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Cabin_Class
0,0,3,male,22.0,1,0,7.2500,NaN,S,na
1,1,1,female,38.0,1,0,71.2833,C85,C,C
2,1,3,female,26.0,0,0,7.9250,NaN,S,na
3,1,1,female,35.0,1,0,53.1000,C123,S,C
4,0,3,male,35.0,0,0,8.0500,NaN,S,na


In [9]:
se = LabelEncoder().fit(df['Sex'])
ee = LabelEncoder().fit(df['Embarked'])
ce = LabelEncoder().fit(df['Cabin_Class'])

In [10]:
df.drop('Cabin', axis=1, inplace=True)

In [11]:
model = XGBClassifier(max_depth=len(df.columns))

In [13]:
df['Sex'] = se.transform(df['Sex'])
df['Embarked'] = ee.transform(df['Embarked'])
df['Cabin_Class'] = ce.transform(df['Cabin_Class'])

In [14]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Cabin_Class
0,0,3,1,22.0,1,0,7.2500,2,8
1,1,1,0,38.0,1,0,71.2833,0,2
2,1,3,0,26.0,0,0,7.9250,2,8
3,1,1,0,35.0,1,0,53.1000,2,2
4,0,3,1,35.0,0,0,8.0500,2,8


In [15]:
df.dropna(inplace=True)

In [16]:
X = df.drop('Survived', axis=1)
y = df['Survived']

In [17]:
model = XGBClassifier()

In [18]:
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [20]:
X_train, X_test, y_train, y_test = X_train.as_matrix(), X_test.as_matrix(), y_train.as_matrix(), y_test.as_matrix()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



In [23]:
X_train

array([[ 1.    ,  1.    , 28.    , ..., 47.1   ,  2.    ,  8.    ],
       [ 3.    ,  0.    , 30.    , ...,  8.6625,  2.    ,  8.    ],
       [ 3.    ,  1.    , 22.    , ...,  8.05  ,  2.    ,  8.    ],
       ...,
       [ 2.    ,  1.    , 50.    , ..., 13.    ,  2.    ,  8.    ],
       [ 3.    ,  1.    , 21.    , ...,  7.7333,  1.    ,  8.    ],
       [ 1.    ,  1.    , 47.    , ..., 25.5875,  2.    ,  4.    ]])

In [22]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [24]:
preds = model.predict(X_test)

In [25]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [26]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.79      0.91      0.85       123
           1       0.85      0.68      0.76        92

    accuracy                           0.81       215
   macro avg       0.82      0.80      0.80       215
weighted avg       0.82      0.81      0.81       215



In [27]:
import pickle

In [28]:
pickle.dump(model, open('./model_rachel.pkl', 'wb'))

# LIME

In [ ]:
import lime
import lime.lime_tabular

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(training_data=X_train, 
                                                   mode='classification',  
                                                   feature_names=X.columns, class_names=[0, 1])

In [ ]:
X.columns

In [ ]:
t = X_test[5]

In [ ]:
exp = explainer.explain_instance(t, model.predict_proba, num_features=8)

In [ ]:
exp.show_in_notebook()

In [ ]:
# First Plot
model.predict_proba([t])[0]

In [ ]:
pred = model.predict_proba([t])
pred

## Viz 1

In [ ]:
def generate_probability_chart(pred):
    """
    Generates a bar chart showing the probability of each class
    :param pred: output of predict_proba function
    """
    # convert passed predictions to numpy array
    npred = np.array(pred)
    
    # grabs the number of classes
    if len(npred.shape) < 2:  # fix for instance where a single list is passed
        npred = npred.reshape(1, -1)
    
    num_classes = npred.shape[1]
    npred = pred.reshape(num_classes,)
    
    layout = go.Layout(title='Class Probabilities')
    fig_ = go.Figure(layout=layout)
    
    for c in range(num_classes):
        fig_.add_trace(go.Bar(
            x = [f'class_{c}'],
            y = [npred[c]],
            name=f'class_{c}', 
        ))
        
    return fig_

In [ ]:
iplot(
    generate_probability_chart(pred)
)

In [ ]:
exp.local_exp

## Viz 2

In [ ]:
def local_exp_df(exp_, feature_names=X.columns):
    """
    Given a lime_explainer.local_exp object returns the cleaned version in a dataframe
    :param exp: lime_explainer.local_exp result
    """
    new_exp = dict(exp_)
    p2g = {}

    for k in new_exp.keys():
        p2g[feature_names[k]] = new_exp[k]
        
    p2g = pd.DataFrame(p2g, index=[0]).T.reset_index()
    p2g.columns = ['feature', 'value']
    return p2g.sort_values(by='value')

lexp = local_exp_df(exp.local_exp[1])
lexp

In [ ]:
def local_feature_importance(data, model, feature_names=X.columns):
    """
    Given a row of data to be evaluate and a model to do the evaluation with, 
    returns a plot showing the local feature importance of each feature
    
    :param data: array of data, output of .predict()
    """
    xdata = np.array(data)
    num_features = len(xdata)
    
    exp = explainer.explain_instance(xdata, model.predict_proba, num_features=num_features)
    new_exp = dict(exp.local_exp[1])
    
    p2g = {}

    for k in new_exp.keys():
        p2g[feature_names[k]] = new_exp[k]
        
    viz = go.Bar(
        x = list(p2g.values()),
        y = list(p2g.keys()),
        name='feature values', 
        orientation='h'
    )

    layout = go.Layout(title='Feature Importance')
    fig = go.Figure([viz], layout)
        
    return fig

In [ ]:
iplot(
    local_feature_importance(t, model=model)
)

In [ ]:
# Third Visual

In [ ]:
exp.domain_mapper.feature_names

In [ ]:
exp.domain_mapper.feature_values

In [ ]:
def feature_values(data, model, feature_names=X.columns):
    """
    Given a row of data to be evaluate and a model to do the evaluation with, 
    returns a plot showing the feature values
    
    :param data: array of data, output of .predict()
    """
    xdata = np.array(data)
    num_features = len(xdata)
    
    exp = explainer.explain_instance(xdata, model.predict_proba, num_features=num_features)
    
    feature_names = exp.domain_mapper.feature_names
    feature_values = exp.domain_mapper.feature_values
    
    table = go.Figure(
        go.Table(
            header=dict(values=['Feature', 'Value']),
            cells=dict(values=[feature_names, feature_values])
        )
    )
    return table
    

In [ ]:
feature_values(t, model)

In [ ]:
class Lime(object):
    
    def __init__(self, model, feature_names, modeling_task=None, classes=None):
        self.model = model
        self.feature_names = feature_names
        self.modeling_task = 'classification' if modeling_task is None else 'regression'
        self.classes = None
        self.num_classes = None
        
        if self.modeling_task == 'classification':
            self.classes = np.array(classes)
            self.num_classes = self.classes.shape[0]
        
            
        
        
        
    def generate_probability_chart(self, data):
        """
        Generates a bar chart showing the probability of each class
        :param pred: output of predict_proba function
        """
        # convert passed predictions to numpy array
        xdata = np.array(data)

        # grabs the number of classes
        if len(xdata.shape) < 2:  # fix for instance where a single list is passed
            xdata = xdata.reshape(1, -1)

        pred = self.model.predict_proba(xdata)
        
        num_classes = pred.shape[1]
        npred = pred.reshape(num_classes,)

        layout = go.Layout(title='Class Probabilities')
        fig_ = go.Figure(layout=layout)

        for c in range(num_classes):
            if {self.classes[c]}:
                fig_.add_trace(go.Bar(
                    x = [f'{self.classes[c]}'],
                    y = [npred[c]],
                    name=f'{self.classes[c]}', 
                ))
            else:
                fig_.add_trace(go.Bar(
                    x = [f'class_{c}'],
                    y = [npred[c]],
                    name=f'class_{c}', 
                ))
                
        return fig_

    
    def local_feature_importance(self, data):
        """
        Given a row of data to be evaluate and a model to do the evaluation with, 
        returns a plot showing the local feature importance of each feature

        :param data: array of data, output of .predict()
        """
        xdata = np.array(data)
        num_features = xdata.shape[0]

        exp = explainer.explain_instance(xdata, self.model.predict_proba, num_features=num_features)
        new_exp = dict(exp.local_exp[1])

        p2g = {}

        for k in new_exp.keys():
            p2g[self.feature_names[k]] = new_exp[k]
            
        colors = ['rgb(255,92,92)' if val < 0 else 'rgb(92,92,255)' for val in p2g.values()]

        viz = go.Bar(
            x = list(p2g.values()),
            y = list(p2g.keys()),
            name='feature values', 
            orientation='h',
            marker=dict(color=colors)
        )

        layout = go.Layout(title='Feature Importance')
        fig = go.Figure([viz], layout)

        return fig


    def feature_values(self, data):
        """
        Given a row of data to be evaluate and a model to do the evaluation with, 
        returns a plot showing the feature values

        :param data: array of data, output of .predict()
        """
        xdata = np.array(data)
        num_features = len(xdata)

        exp = explainer.explain_instance(xdata, self.model.predict_proba, num_features=num_features)

        feature_names = exp.domain_mapper.feature_names
        feature_values = exp.domain_mapper.feature_values

        table = go.Figure(
            go.Table(
                header=dict(values=['Feature', 'Value']),
                cells=dict(values=[feature_names, feature_values])
            )
        )
        return table


In [ ]:
l = Lime(model=model, feature_names=X.columns, classes=['not_survived', 'survived'])

In [ ]:
X_test[0]

In [ ]:
l.local_feature_importance(X_test[4])

In [ ]:
l.generate_probability_chart(X_test[2])

In [ ]:
l.feature_values(X_test[2])

# Yellowbricks

In [ ]:
visualizer = ParallelCoordinates()
visualizer.fit_transform(X, y)
visualizer.poof()

# ParallelCoordinates

In [ ]:
visualizer = ParallelCoordinates()
visualizer.fit_transform(X, y)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df.corrwith(df['Survived']).values.reshape(1, -1), 
            xticklabels=df.columns,
            yticklabels=['Survived'],
           annot=True, center=True)

In [ ]:
visualizer = ClassificationReport(model)

visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.poof()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
class CustomVisuals(object):
    
    def __init__(self, model, feature_names, classes):
        self.model = model
        self.feature_names = feature_names
        self.classes = classes
        
    def generate_classification_report(self, X_test, y_test, 
                                       zmin=0, zmax=1, annot=False, 
                                       **kwargs):
        """
        Given the test data sets, produces a classification report and returns a plotly
        Heatmap figure.
        
        :param X_test: The withheld input features from the training data set.
        :param y_test: The withheld output target from the training data set.
        :param zmin: (optional) sets scale minimum. Defaults to 0.
        :param zmax: (optional) sets scale maxmimum. Defaults to 1.
        """
        cr = classification_report(y_test, self.model.predict(X_test), output_dict=True)
        cr = pd.DataFrame(cr)
        
        fig_x = self.classes + ['accuracy', 'macro avg', 'weighted avg']
        fig_y = cr.index
        z_text = cr.values
        
        fig = go.Figure(
            go.Heatmap(
                z = cr.values,
                x = fig_x,
                y = fig_y,
                zmin = zmin if zmin is not None else zmin,
                zmax = zmax if zmax is not None else zmax,
                text = None if annot == False else z_text,
                **kwargs
            )
        )
        return fig
    def generate_confusion_matrix(self, X_test, y_test, **kwargs):
        """
        Given the test data sets, produces a confusion matrixs and returns a 
        plotly Heatmap figure.
        
        :param X_test: The withheld input features from the training data set.
        :param y_test: The withheld output target from the training data set.
        """
        
        cm = confusion_matrix(y_test, self.model.predict(X_test))
        
        fig_x = self.classes
        fig_y = self.classes
        fig_z = cm

        hovertext = list()
        for yi, yy in enumerate(fig_y):
            hovertext.append(list())
            for xi, xx in enumerate(fig_x):
                hovertext[-1].append(f'Predicted: {xx}<br />Actual: {yy}<br />Count: {fig_z[yi][xi]}')
      
        fig = go.Figure(
            go.Heatmap(
                z = fig_z, 
                x = fig_x,
                y = fig_y,
                hoverinfo='text',
                text=hovertext,
                **kwargs
            ),
            go.Layout(
                xaxis=go.layout.XAxis(
                    title=go.layout.xaxis.Title(text='Predicted', font=dict(size=24))
                ),
                yaxis=go.layout.YAxis(
                    title=go.layout.yaxis.Title(text='Actuals', font=dict(size=24))
                )
            
            )
        )
        return fig
    
    

    def generate_rank_2d(self, X, algorithm='pearson', **kwargs):
        """
        Given the entire (train+test) input features, returns a plotly
        Heatmap figure showing the feature x feature correlation.
        
        :param X: the input features to the model
        :param algorithm: the algorithm to calculate the importance with 
                          (pearson, covariance, spearman, kendalltau)
        """
        
        visualizer = Rank2D(algorithm=algorithm)
        visualizer.fit_transform(X)
        
        #grabbing feature shape
        feats = ranks_.shape[0]
        
        # zero-ing out one of the diagonals features
        iu = np.triu_indices(feats, )
        ranks_[iu] = 0
        
        fig = go.Figure(
            go.Heatmap(
                z = ranks_,
                x = visualizer.features_,
                y = np.flip(visualizer.features_), 
                **kwargs
                
            )
        )
        
        return fig
    
    
    def generate_rank_1d(self, X, y, algorithm='shapiro', **kwargs):
        """
        Given the entire (train+test) input and target features, returns 
        a plotly figure showing the feature correlation.
        
        :param X: the input features to the model
        :param y: the target feature
        """
        
        visualizer = Rank1D(algorithm='shapiro')

        visualizer.fit(X, y)           # Fit the data to the visualizer
        visualizer.transform(X)        # Transform the data
        
        fig = go.Figure(
            go.Bar(
                x = visualizer.ranks_,
                y = visualizer.features_,
                orientation='h'
            )
        )
        return fig
        
    def generate_rad_viz(self, X, y, **kwargs):
        """
        Given the entire feature-space, X, and target-space, y, 
        generates a RadViz plotly object.
        
        :param X: the input features to the model.
        :param y: the target variables to the model.
        """
        
        # grabbing rows, columns, to later build out the radviz data set
        nrows, ncols = X.shape
        to_plot = {label: [[], []] for label in yb.classes}
        
        # locations of where to plot the feature names.
        class_points = np.array(
            [
                (np.cos(t), np.sin(t)) for t in [2.0 * np.pi * (i / float(ncols)) for i in range(ncols)]
            ]
        )
        
        # converting from Data Frame to numpy array if need be
        sc = MinMaxScaler(feature_range=(0,1))    
        X_scaled = sc.fit_transform(X)
        
        # Compute the locations of the scatter plot for each class
        for i, row in enumerate(X_scaled):
            row_ = np.repeat(np.expand_dims(row, axis=1), 2, axis=1)
            xy = (class_points * row_).sum(axis=0) / row.sum()
            #label = self.classes[j] for j in y
            label = self.classes[y.values[i]]
            #label = 'not_survived' if y.values[i] == 0 else 'survived'  #self._label_encoder[y[i]]
            to_plot[label][0].append(xy[0])
            to_plot[label][1].append(xy[1])

        layout = go.Layout(yaxis = dict(
                            scaleanchor = "x",
                            #scaleratio = 1,
                            range=[-1.1, 1.1]
                        ),
                        xaxis=dict(
                            range=[-1.1, 1.1],
                            #scaleratio = 1
                        ),
                )

        fig = go.Figure(layout=layout)
            
        for lab in self.classes:
            trace = go.Scatter(
                x = to_plot[lab][0],
                y = to_plot[lab][1],
                mode = 'markers', 
                name = f'{lab}'
            )
            fig.add_trace(trace)
        

        class_trace = go.Scatter(
            x = [class_points[i][0] for i in range(len(class_points))] + [class_points[0][0]],
            y = [class_points[i][1] for i in range(len(class_points))] + [class_points[0][1]],
            name = 'Features',
            mode = 'lines+text',
            text = [self.feature_names[i] for i in range(len(self.feature_names))]
        )
        
        fig.add_trace(class_trace)

        return fig
    
    def generate_roc_auc(self, X_train, y_train, X_test, y_test, **kwargs):
        """
        Given the training and testing sets, computes the ROC AUC metrics
        for the given model and returns a ROC AUC plotly figure.
        
        :param X_train: the training feature set.
        :param y_train: the training target set.
        :param X_test: the testing feature set.
        :param y_test: the testing target set.
        """
        
        visualizer = ROCAUC(self.model, classes=self.classes)

        visualizer.fit(X_train, y_train)
        visualizer.score(X_test, y_test)
        
        roc_data = visualizer.tpr
        
        layout = go.Layout(yaxis = dict(
                                scaleratio = 1,
                                range=[-.1, 1]
                            ),
                            xaxis=dict(
                                range=[-.1, 1],
                                #scaleratio = 1
                            ),
                        )

        fig = go.Figure(layout=layout)

        for tr in roc_data.keys():
            trace = go.Scatter(
                x = [i / len(roc_data[tr]) for i in range(len(roc_data[tr]))],
                y = roc_data[tr],
                name = f'{tr}' if type(tr) != int else f'{yb.classes[tr]}',
                line = dict(shape = 'hv')
            )
            fig.add_trace(trace)

        lin_line = go.Scatter(
            x = [0,1],
            y = [0,1],
            name = 'linear_line',
            line = dict(dash='dash')
        )

        fig.add_trace(lin_line)
        return fig
    
    def generate_pca_2d(self, X, y, scale=True, **kwargs):
        """
        Given the entire feature and target sets, performs a 2D Principal
        Component Analysis and returns a plotly figure
        
        :param X: model feature set.
        :param y: model target set.
        :param scale: bool indicating if you want to perform Standard Scaling.
        """
        
        if scale:
            X = StandardScaler().fit_transform(X)

        model = PCA(n_components = 2)
        model.fit(X)
        features = model.transform(X) 
        
        layout = go.Layout(
            xaxis=dict(title='PC<sub>1</sub>'),
            yaxis=dict(title='PC<sub>2</sub>')
        )
        
        fig = go.Figure(layout=layout)
        
        for class_ in range(len(self.classes)):
            temp_x, temp_y = features[y == class_].T
            
            trace = go.Scatter(
                x = temp_x,
                y = temp_y,
                name = f'{self.classes[class_]}',
                mode = 'markers'
            )
            
            fig.add_trace(trace)
    
        return fig
    
    
    def generate_pca_3d(self, X, y, scale=True, **kwargs):
        """
        Given the entire feature and target sets, performs a 3D Principal
        Component Analysis and returns a plotly figure
        
        :param X: model feature set.
        :param y: model target set.
        :param scale: bool indicating if you want to perform Standard Scaling.
        """
        
        if scale:
            X = StandardScaler().fit_transform(X)

        model = PCA(n_components = 3)
        model.fit(X)
        features = model.transform(X) 
        
        fig = go.Figure()
        
        for class_ in range(len(self.classes)):
            temp_x, temp_y, temp_z = features[y == class_].T
            
            trace = go.Scatter3d(
                x = temp_x,
                y = temp_y,
                z = temp_z,
                name = f'{self.classes[class_]}',
                mode = 'markers'
            )
            
            fig.add_trace(trace)
            
        
        return fig

In [ ]:
cv = CustomVisuals(model=model, feature_names=X.columns, classes=['not_survived', 'survived'])

In [ ]:
cv.generate_classification_report(X_test, y_test, colorscale='Viridis', annot=True)

## confusion matrix

In [ ]:
cv.generate_confusion_matrix(X_test, y_test, colorscale='Viridis')

# Yellowbrick

In [ ]:
visualizer = Rank2D(algorithm='pearson')
visualizer.fit_transform(X)

In [ ]:
## values
ranks_ = visualizer.ranks_

In [ ]:
visualizer.ranking_methods

In [ ]:
class YellowBrick(object):
    
    def __init__(self, model, feature_names, classes):
        self.model = model
        self.feature_names = feature_names
        self.classes = classes
       
    
    def generate_rank_2d(self, X, algorithm='pearson', **kwargs):
        """
        Given the entire (train+test) input features, returns a plotly
        Heatmap figure showing the feature x feature correlation.
        
        :param X: the input features to the model
        :param algorithm: the algorithm to calculate the importance with 
                          (pearson, covariance, spearman, kendalltau)
        """
        
        visualizer = Rank2D(algorithm=algorithm)
        visualizer.fit_transform(X)
        
        #grabbing feature shape
        feats = ranks_.shape[0]
        
        # zero-ing out one of the diagonals features
        iu = np.triu_indices(feats, )
        ranks_[iu] = 0
        
        fig = go.Figure(
            go.Heatmap(
                z = ranks_,
                x = visualizer.features_,
                y = np.flip(visualizer.features_), 
                **kwargs
                
            )
        )
        
        return fig
    
    
    def generate_rank_1d(self, X, y, algorithm='shapiro', **kwargs):
        """
        Given the entire (train+test) input and target features, returns 
        a plotly figure showing the feature correlation.
        
        :param X: the input features to the model
        :param y: the target feature
        """
        
        visualizer = Rank1D(algorithm='shapiro')

        visualizer.fit(X, y)           # Fit the data to the visualizer
        visualizer.transform(X)        # Transform the data
        
        fig = go.Figure(
            go.Bar(
                x = visualizer.ranks_,
                y = visualizer.features_,
                orientation='h'
            )
        )
        return fig
        
    def generate_rad_viz(self, X, y, **kwargs):
        """
        Given the entire feature-space, X, and target-space, y, 
        generates a RadViz plotly object.
        
        :param X: the input features to the model.
        :param y: the target variables to the model.
        """
        
        # grabbing rows, columns, to later build out the radviz data set
        nrows, ncols = X.shape
        to_plot = {label: [[], []] for label in yb.classes}
        
        # locations of where to plot the feature names.
        class_points = np.array(
            [
                (np.cos(t), np.sin(t)) for t in [2.0 * np.pi * (i / float(ncols)) for i in range(ncols)]
            ]
        )
        
        # converting from Data Frame to numpy array if need be
        sc = MinMaxScaler(feature_range=(0,1))    
        X_scaled = sc.fit_transform(X)
        
        # Compute the locations of the scatter plot for each class
        for i, row in enumerate(X_scaled):
            row_ = np.repeat(np.expand_dims(row, axis=1), 2, axis=1)
            xy = (class_points * row_).sum(axis=0) / row.sum()
            #label = self.classes[j] for j in y
            label = self.classes[y.values[i]]
            #label = 'not_survived' if y.values[i] == 0 else 'survived'  #self._label_encoder[y[i]]
            to_plot[label][0].append(xy[0])
            to_plot[label][1].append(xy[1])

        layout = go.Layout(yaxis = dict(
                            scaleanchor = "x",
                            #scaleratio = 1,
                            range=[-1.1, 1.1]
                        ),
                        xaxis=dict(
                            range=[-1.1, 1.1],
                            #scaleratio = 1
                        ),
                )

        fig = go.Figure(layout=layout)
            
        for lab in self.classes:
            trace = go.Scatter(
                x = to_plot[lab][0],
                y = to_plot[lab][1],
                mode = 'markers', 
                name = f'{lab}'
            )
            fig.add_trace(trace)
        

        class_trace = go.Scatter(
            x = [class_points[i][0] for i in range(len(class_points))] + [class_points[0][0]],
            y = [class_points[i][1] for i in range(len(class_points))] + [class_points[0][1]],
            name = 'Features',
            mode = 'lines+text',
            text = [self.feature_names[i] for i in range(len(self.feature_names))]
        )
        
        fig.add_trace(class_trace)

        return fig
    
    def generate_roc_auc(self, X_train, y_train, X_test, y_test, **kwargs):
        """
        Given the training and testing sets, computes the ROC AUC metrics
        for the given model and returns a ROC AUC plotly figure.
        
        :param X_train: the training feature set.
        :param y_train: the training target set.
        :param X_test: the testing feature set.
        :param y_test: the testing target set.
        """
        
        visualizer = ROCAUC(self.model, classes=self.classes)

        visualizer.fit(X_train, y_train)
        visualizer.score(X_test, y_test)
        
        roc_data = visualizer.tpr
        
        layout = go.Layout(yaxis = dict(
                                scaleratio = 1,
                                range=[-.1, 1]
                            ),
                            xaxis=dict(
                                range=[-.1, 1],
                                #scaleratio = 1
                            ),
                        )

        fig = go.Figure(layout=layout)

        for tr in roc_data.keys():
            trace = go.Scatter(
                x = [i / len(roc_data[tr]) for i in range(len(roc_data[tr]))],
                y = roc_data[tr],
                name = f'{tr}' if type(tr) != int else f'{yb.classes[tr]}',
                line = dict(shape = 'hv')
            )
            fig.add_trace(trace)

        lin_line = go.Scatter(
            x = [0,1],
            y = [0,1],
            name = 'linear_line',
            line = dict(dash='dash')
        )

        fig.add_trace(lin_line)
        return fig
    
    def generate_pca_2d(self, X, y, scale=True, **kwargs):
        """
        Given the entire feature and target sets, performs a 2D Principal
        Component Analysis and returns a plotly figure
        
        :param X: model feature set.
        :param y: model target set.
        :param scale: bool indicating if you want to perform Standard Scaling.
        """
        
        if scale:
            X = StandardScaler().fit_transform(X)

        model = PCA(n_components = 2)
        model.fit(X)
        features = model.transform(X) 
        
        layout = go.Layout(
            xaxis=dict(title='PC<sub>1</sub>'),
            yaxis=dict(title='PC<sub>2</sub>')
        )
        
        fig = go.Figure(layout=layout)
        
        for class_ in range(len(self.classes)):
            temp_x, temp_y = features[y == class_].T
            
            trace = go.Scatter(
                x = temp_x,
                y = temp_y,
                name = f'{self.classes[class_]}',
                mode = 'markers'
            )
            
            fig.add_trace(trace)
    
        return fig
    
    
    def generate_pca_3d(self, X, y, scale=True, **kwargs):
        """
        Given the entire feature and target sets, performs a 3D Principal
        Component Analysis and returns a plotly figure
        
        :param X: model feature set.
        :param y: model target set.
        :param scale: bool indicating if you want to perform Standard Scaling.
        """
        
        if scale:
            X = StandardScaler().fit_transform(X)

        model = PCA(n_components = 3)
        model.fit(X)
        features = model.transform(X) 
        
        fig = go.Figure()
        
        for class_ in range(len(self.classes)):
            temp_x, temp_y, temp_z = features[y == class_].T
            
            trace = go.Scatter3d(
                x = temp_x,
                y = temp_y,
                z = temp_z,
                name = f'{self.classes[class_]}',
                mode = 'markers'
            )
            
            fig.add_trace(trace)
            
        
        return fig

In [ ]:
yb = YellowBrick(model=model, feature_names=X.columns, classes=['not_survived', 'survived'])

In [ ]:
cv.generate_rank_1d(X, y)

In [ ]:
cv.generate_rank_2d(X)

In [ ]:
cv.generate_rad_viz(X, y)

In [ ]:
cv.generate_roc_auc(X_train, y_train, X_test, y_test)

In [ ]:
cv.generate_pca_2d(X, y)

In [ ]:
cv.generate_pca_3d(X, y)

In [ ]:
import pickle

In [ ]:
pickle.dump(model, open('../models/model.pkl', 'wb'))

In [ ]:
np.save('../models/X_train.npy', X_train)
np.save('../models/X_test.npy', X_test)
np.save('../models/y_train.npy', y_train)
np.save('../models/y_test.npy', y_test)